In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")

In [2]:
vm = input()
if vm.lower()=='yes':
    vm=True
else:
    vm=False

yes


In [3]:
train = input()
if train.lower()=='yes':
    train=True
else:
    train=False

yes


In [4]:
if vm:
    path='/mnt/cephfs/ml_data/mc_2021/processed_data/ProcessedTrainReal/'
    data_real = pd.read_csv('{}ProcessedTrain_1M.csv.gz'.format(path))
else:
    data_real = pd.read_csv('processed_data/ProcessedTrainReal/ProcessedTrain_1M.csv.gz')

In [5]:
data_real = data_real[data_real['edepR'] < 17.2]

In [6]:
data_real.shape

(1147444, 111)

In [ ]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

if train:
    xgbreg = XGBRegressor(
        max_depth=11,
        learning_rate=0.08,
        n_estimators=3000,
        random_state=22,
    )

    scores = cross_val_score(
        xgbreg,
        data_real.iloc[:, :-5],
        data_real.iloc[:, -5],
        cv=5,
        n_jobs=5,
        verbose=10,
        scoring='neg_mean_absolute_percentage_error'
    )
    
    result = 100 * np.array([-scores.mean(), -scores.std()**0.5])
    np.savez_compressed('feature_selection/all_features_metric.npz', a=result)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.


In [ ]:
print(result)

In [ ]:
from sklearn.metrics import mean_squared_error

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
all_features_metric = np.load('feature_selection/all_features_metric.npz', allow_pickle=True)['a'][0]
eps = np.load('feature_selection/all_features_metric.npz', allow_pickle=True)['a'][1]

opt_features = []
current_metrics = []
current_metric = 100

features = data_real.iloc[:, :-5].columns
while abs(all_features_metric - current_metric) > eps:
    metrics = []
    for feature in tqdm(features, "Features loop"):
        xgbreg = XGBRegressor(
            max_depth=11,
            learning_rate=0.08,
            n_estimators=3000,
            random_state=22,
        )
        
        scores = cross_val_score(
            xgbreg,
            data_real.iloc[:, :-5][opt_features+[feature]],
            data_real.iloc[:, -5],
            cv=5,
            n_jobs=5,
            verbose=10,
            scoring='neg_mean_absolute_percentage_error'
        )
        
        metric = -100*scores.mean()
        metrics.append(metric)

    best_metric_ind = np.argmin(metrics)
    current_metric = metrics[best_metric_ind]
    current_metrics.append(current_metric)
    opt_features.append(features[best_metric_ind])
    features = features.drop(features[best_metric_ind])

    print(current_metrics)
    print(opt_features)
    
    np.savez_compressed('feature_selection/opt_features.npz', a=np.array(opt_features))
    np.savez_compressed('feature_selection/current_metrics.npz', a=np.array(current_metrics))

In [ ]:
np.savez_compressed('feature_selection/opt_features.npz', a=np.array(opt_features))
np.savez_compressed('feature_selection/current_metrics.npz', a=np.array(current_metrics))